In [37]:
# Cross-Attention Fusion of GRU + Offseason Features

import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Dropout
from sklearn.preprocessing import StandardScaler

In [38]:
# ---------------------------
# Load Data
# ---------------------------
gru_df = pd.read_csv("gru_team_embeddings_by_week.csv")
offseason_df = pd.read_csv("offseason_2024_labeled.csv")
nfl_sched_2024 = pd.read_excel("nfl_2024_schedule.xlsx")
hist_df = pd.read_csv("final_hist_data.csv")  # historical dataset with true feature names

offseason_df['Team'] = ['CRD', 'ATL', 'RAV', 'BUF', 'CAR', 'CHI', 'CIN', 'CLE', 'DAL',
              'DEN', 'DET', 'GNB', 'HTX', 'CLT', 'JAX', 'KAN', 'RAI', 'SDG',
              'RAM', 'MIA', 'MIN', 'NWE', 'NOR', 'NYG', 'NYJ', 'PHI', 'PIT',
              'SFO', 'SEA', 'TAM', 'OTI', 'WAS'] 

# ---------------------------
# Scale numeric, preserve categorical
# ---------------------------
categorical_cols = ["New Coach", "New QB"]
numeric_cols = [c for c in offseason_df.columns 
                if c not in ["Improvement?", "Team"] + categorical_cols]

scaler = StandardScaler()
offseason_scaled = offseason_df.copy()
offseason_scaled[numeric_cols] = scaler.fit_transform(offseason_scaled[numeric_cols])
offseason_scaled[categorical_cols] = offseason_df[categorical_cols]  # untouched

feature_cols = numeric_cols + categorical_cols

In [39]:
# ---------------------------
# Rename GRU embeddings to match hist features
# ---------------------------
gru_cols = [c for c in gru_df.columns if c.startswith("emb_")]
hist_feature_names = [c for c in hist_df.columns if c not in ["Seasonweek", "Result", "Team"]]

assert len(gru_cols) == len(hist_feature_names), \
    f"Mismatch: GRU embeddings = {len(gru_cols)}, hist features = {len(hist_feature_names)}"

gru_feature_map = {old: new for old, new in zip(gru_cols, hist_feature_names)}
gru_df = gru_df.rename(columns=gru_feature_map)
gru_cols = hist_feature_names  # now GRU embeddings have meaningful names

all_feature_names = gru_cols + feature_cols

In [40]:
# ---------------------------
# Merge off-season features into GRU
# ---------------------------
gru_with_feats = gru_df.merge(
    offseason_scaled[["Team"] + feature_cols],
    left_on="team", right_on="Team", how="left"
).drop(columns=["Team"])

print("Shape with GRU + offseason features:", gru_with_feats.shape)

Shape with GRU + offseason features: (1598, 34)


In [41]:
# ---------------------------
# Build Games List
# ---------------------------
games = []
for _, row in nfl_sched_2024.iterrows():
    games.append({
        "week": int(row["Seasonweek"]),
        "away": row["Away Team"],
        "home": row["Home Team"],
        "winner": row["Winner"]
    })
print(f"Loaded {len(games)} games.")

Loaded 272 games.


In [42]:
# ---------------------------
# Helper: Team vector
# ---------------------------
def get_team_vector(team, week):
    """Concatenate GRU embeddings + offseason features for a given team/week."""
    row = gru_with_feats[(gru_with_feats["team"] == team) &
                         (gru_with_feats["seasonweek"] == week)]
    if row.empty:
        return None
    gru_emb = row[gru_cols].values.flatten()
    feats = row[feature_cols].values.flatten()
    return np.concatenate([gru_emb, feats])

In [43]:
# ---------------------------
# Cross-Attention Layer
# ---------------------------
class CrossAttentionLayer(layers.Layer):
    def __init__(self, embed_dim, num_heads=2, ff_dim=64, rate=0.1):
        super().__init__()
        self.attn = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = tf.keras.Sequential([
            Dense(ff_dim, activation="relu"),
            Dense(embed_dim),
        ])
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(rate)
        self.dropout2 = Dropout(rate)

    def call(self, query, value, training=False):
        attn_output = self.attn(query, value)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(query + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

In [44]:
# ---------------------------
# Build Matchup Embeddings
# ---------------------------
attended_rows = []
for g in games:
    week = g["week"]
    away_team, home_team = g["away"], g["home"]

    away_vec = get_team_vector(away_team, week)
    home_vec = get_team_vector(home_team, week)

    if away_vec is None or home_vec is None:
        print(f"Skipping {away_team} vs {home_team} (week {week}) — missing data")
        continue

    away_in = tf.expand_dims(tf.expand_dims(away_vec, 0), 0)
    home_in = tf.expand_dims(tf.expand_dims(home_vec, 0), 0)

    embed_dim = away_vec.shape[0]
    cross_layer = CrossAttentionLayer(embed_dim)

    away_attended = tf.squeeze(cross_layer(away_in, home_in)).numpy()
    home_attended = tf.squeeze(cross_layer(home_in, away_in)).numpy()

    attended_rows.append({
        "week": week,
        "away_team": away_team,
        "home_team": home_team,
        "winner": g["winner"],
        **{f"A_{name}": v for name, v in zip(all_feature_names, away_attended)},
        **{f"B_{name}": v for name, v in zip(all_feature_names, home_attended)},
    })

attended_df = pd.DataFrame(attended_rows)
attended_df.to_csv("crossatt_gru_offseason_scaled_named.csv", index=False)

print("✅ Saved:", attended_df.shape)
print(attended_df.head())

✅ Saved: (272, 68)
     week away_team home_team winner    A_Rate  A_Def Passer Rate  \
0  202401       RAV       KAN    KAN  0.705361           0.419476   
1  202401       GNB       PHI    PHI -0.372874          -0.751220   
2  202401       PIT       ATL    PIT -2.269074          -1.371742   
3  202401       CRD       BUF    BUF  0.889368          -0.493526   
4  202401       CAR       NOR    NOR -0.076837          -0.796135   

   A_Def Rush Att  A_Rush Att  A_TD allowed  A_Def Cmp%  ...      B_QB  \
0       -1.393290   -0.370963     -0.774228    0.027848  ... -0.579894   
1       -0.798091    0.293588     -0.279107   -0.215493  ...  0.472573   
2       -0.005273    1.462215     -0.596726   -0.246090  ... -0.600479   
3        1.628955   -0.152389      0.738619   -1.696822  ...  2.945529   
4        1.242722   -0.889176     -0.900348   -0.495276  ... -1.289932   

       B_OL      B_LB      B_DE      B_ST      B_DL      B_RB      B_WR  \
0  2.130383  0.490881 -1.683697 -1.286602  0.6